# ChatGPT

"ChatGPT" like examples.  Adapted from
[LangChain](https://langchain.readthedocs.io/en/latest/modules/memory/examples/chatgpt_clone.html)'s
version of this [blog
post](https://www.engraved.blog/building-a-virtual-machine-inside/).

In [1]:
from dataclasses import dataclass
from typing import List, Tuple
import minichain
from IPython.display import Markdown, display

In [2]:
import warnings
warnings.filterwarnings("ignore")

Generic stateful Memory

In [3]:
MEMORY = 2

In [4]:
@dataclass
class State:
    memory: List[Tuple[str, str]]
    human_input: str = ""

    def push(self, response: str) -> "State":
        memory = self.memory if len(self.memory) < MEMORY else self.memory[1:]
        return State(memory + [(self.human_input, response)])

Chat prompt with memory

In [5]:
class ChatPrompt(minichain.TemplatePrompt):
    template_file = "chatgpt.pmpt.tpl"
    def parse(self, out: str, inp: State) -> State:
        result = out.split("Assistant:")[-1]
        return inp.push(result)

In [6]:
fake_human = [
    "I want you to act as a Linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in English I will do so by putting text inside curly brackets {like this}. My first command is pwd.",
    "ls ~",
    "cd ~",
    "{Please make a file jokes.txt inside and put some jokes inside}",
    """echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py""",
    """echo -e "print(list(filter(lambda x: all(x%d for d in range(2,x)),range(2,3**10)))[:10])" > run.py && python3 run.py""",
    """echo -e "echo 'Hello from Docker" > entrypoint.sh && echo -e "FROM ubuntu:20.04\nCOPY entrypoint.sh entrypoint.sh\nENTRYPOINT [\"/bin/sh\",\"entrypoint.sh\"]">Dockerfile && docker build . -t my_docker_image && docker run -t my_docker_image""",
    "nvidia-smi"
]

In [7]:
with minichain.start_chain("chatgpt") as backend:
    prompt = ChatPrompt(backend.OpenAI())
    state = State([])
    for t in fake_human:
        state.human_input = t
        display(Markdown(f'**Human:** <span style="color: blue">{t}</span>'))
        state = prompt(state)
        display(Markdown(f'**Assistant:** <span>{state.memory[-1][1]}</span>'))
        display(Markdown(f'--------------'))

**Human:** <span style="color: blue">I want you to act as a Linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in English I will do so by putting text inside curly brackets {like this}. My first command is pwd.</span>

**Assistant:** <span>
```
$ pwd
/
```</span>

--------------

**Human:** <span style="color: blue">ls ~</span>

**Assistant:** <span>
```
$ ls ~
Desktop/ Documents/ Downloads/ Music/ Pictures/ Public/ Templates/ Videos/
```</span>

...

**Assistant:** <span>

```
$ echo -e "echo 'Hello from Docker" > entrypoint.sh
$ echo -e "FROM ubuntu:20.04
COPY entrypoint.sh entrypoint.sh
ENTRYPOINT ["/bin/sh","entrypoint.sh"]">Dockerfile
$ docker build . -t my_docker_image
$ docker run -t my_docker_image
Hello from Docker
```</span>

--------------

**Human:** <span style="color: blue">nvidia-smi</span>

**Assistant:** <span>

```
$ nvidia-smi
Sat May 15 21:45:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P0    28W / 250W |      0MiB /  10240MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|  No running</span>

--------------

In [8]:
ChatPrompt().show(State([("human 1", "output 1"), ("human 2", "output 2") ], "cd ~"),
                    "Text Assistant: Hello")

HTML(html='\n<!-- <link rel="stylesheet" href="https://cdn.rawgit.com/Chalarangelo/mini.css/v3.0.1/dist/mini-default.min.css"> -->\n <main class="container">\n\n<h3>ChatPrompt</h3>\n\n<dl>\n  <dt>Input:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="n">State</span><span class="p">(</span><span class="n">memory</span><span class="o">=</span><span class="p">[(</span><span class="s1">&#39;human 1&#39;</span><span class="p">,</span> <span class="s1">&#39;output 1&#39;</span><span class="p">),</span> <span class="p">(</span><span class="s1">&#39;human 2&#39;</span><span class="p">,</span> <span class="s1">&#39;output 2&#39;</span><span class="p">)],</span> <span class="n">human_input</span><span class="o">=</span><span class="s1">&#39;cd ~&#39;</span><span class="p">)</span>\n</pre></div>\n\n\n  </dd>\n\n  <dt> Full Prompt: </dt>\n  <dd>\n    <details>\n      <summary>Prompt</summary>\n      <p>Assistant is a large language model trained by OpenAI.<br><br>Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.<br><br>Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.<br><br>Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.<br><br><br>Human: human 1<br>AI: output 1<br><br>Human: human 2<br>AI: output 2<br><br><br>Human: <div style=\'color:red\'>cd ~</div><br>Assistant:</p>\n    </details>\n  </dd>\n\n  <dt> Response: </dt>\n  <dd>\n    Text Assistant: Hello\n  </dd>\n\n  <dt>Value:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="n">State</span><span class="p">(</span><span class="n">memory</span><span class="o">=</span><span class="p">[(</span><span class="s1">&#39;human 2&#39;</span><span class="p">,</span> <span class="s1">&#39;output 2&#39;</span><span class="p">),</span> <span class="p">(</span><span class="s1">&#39;cd ~&#39;</span><span class="p">,</span> <span class="s1">&#39; Hello&#39;</span><span class="p">)],</span> <span class="n">human_input</span><span class="o">=</span><span class="s1">&#39;&#39;</span><span class="p">)</span>\n</pre></div>\n\n  </dd>\n</main>\n')

View the run log.

In [9]:
minichain.show_log("chatgpt.log")

3fde6368-ae6a-4383-a4c8-ea3c1879fa23
└── chatgpt/1 ⇒ started 2023-02-27 16:17:10Z ⧖ 32.318s
    └── chatgpt/2 ⇒ succeeded 2023-02-27 16:17:43Z

096b5b44-733c-4282-bcce-28830868a519
└── <class '__main__.ChatPrompt'>/1 ⇒ started 2023-02-27 16:17:19Z ⧖ 3.375s
    ├── <unnamed>
    │   └── Input Function/2/2 ⇒ succeeded 2023-02-27 16:17:19Z
    ├── eliot:destination_failure/3 2023-02-27 16:17:19Z
    │   ├── exception: builtins.AttributeError
    │   ├── message: {"'input'": 'State(memory=[(\'cd ~\', \' \\n```\\n$ cd ~\\n$ pwd\\n/home/username\\n```\'), (\'{Please make a file jokes.txt inside and put some jokes inside}\', \'\\n\\n```\\n$ touch jokes.txt\\n$ echo "Why did the chicken cross the road? To get to the other side!" >> jokes.txt\\n$ echo "What did the fish say when it hit the wall? Dam!" >> jokes.txt\\n$ echo "Why did the scarecrow win the Nobel Prize? Because he was outstanding in his field!" >> jokes.txt\\n```\')], human_input=\'echo -e "x=lambda y:y*5+3;print(\\\'Result:\\\' + 